#### imports

In [ ]:
import tensorflow as tf
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import pathlib
import PIL
import PIL.Image

#### parameters

In [ ]:
# data
data_dir = pathlib.Path('./data')
data = list(data_dir.glob('*/*.*'))
img_height = 227
img_width = 227

# partitioning
test_set_ratio = 0.1
validation_set_ratio = 0.1

# random
random_seed = 42

# keras
batch_size = 32

#### load all data

In [ ]:
ds_all = tf.keras.utils.image_dataset_from_directory(
    batch_size = batch_size,
    directory = data_dir,
    image_size = (img_height, img_width), # resized image dimensions
    seed = random_seed) # ensure known random_seed is used (for consistency)
ds_all_class_names = ds_all.class_names

In [ ]:
print(ds_all)

In [ ]:
img_scale = 0.1
img_col_count = 3
img_row_count = 3

plt.figure(figsize=(img_height * img_scale, img_width * img_scale))
for images, labels in ds_all.take(1):
    for i in range(img_col_count * img_row_count):
        ax = plt.subplot(img_row_count, img_col_count, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(ds_all_class_names[labels[i]])
        plt.axis("off")

#### partition data

In [ ]:
ds_all_size = ds_all.__len__().numpy()
print("ds_all_size        => " + str(ds_all_size))

# use first batches for test and validation as these are full batches (last batch might be partial batch)
ds_test_size       = max(1, int(ds_all_size * test_set_ratio      ))
ds_validation_size = max(1, int(ds_all_size * validation_set_ratio))
print("ds_test_size       => " + str(ds_test_size))
print("ds_validation_size => " + str(ds_validation_size))

# training set becomes rest
ds_training_size = ds_all_size - ds_validation_size - ds_test_size
print("ds_training_size   => " + str(ds_training_size))

In [ ]:
ds_test       = ds_all                                        .take(ds_test_size      )
ds_validation = ds_all.skip(ds_test_size                     ).take(ds_validation_size)
ds_training   = ds_all.skip(ds_test_size + ds_validation_size).take(ds_training_size  )

ds_training_repeated = ds_training.repeat()

# check sizes
print("ds_test      .__len__() => " + str(ds_test      .__len__().numpy()))
print("ds_validation.__len__() => " + str(ds_validation.__len__().numpy()))
print("ds_training  .__len__() => " + str(ds_training  .__len__().numpy()))

#### train model 1

In [ ]:
# get base model
base_model = tf.keras.applications.resnet_v2.ResNet50V2(
    include_top = False,
    input_shape = (img_height, img_width, 3),
    pooling = 'max', # ( None | avg | max )
    weights = 'imagenet') # ( None | imagenet | path to weights file)

for layer in base_model.layers:
    layer.trainable = False

# add convolution layers
model_1 = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(512, 'relu'),
    tf.keras.layers.Dense(256, 'relu'),
    tf.keras.layers.Dense(2, 'softmax')
])

model_1.summary()

In [ ]:
model_1.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['sparse_categorical_accuracy'],
    optimizer = 'adam')

history = model_1.fit(
    ds_training_repeated,
    epochs=20,
    steps_per_epoch=5) # ( None (not with infinite datasets) | int )